Bộ dữ liệu chạy thử nghiệm là bộ dl lấy trong bài báo Improving protein succinylation sites prediction using embeddings from protein language model
link: https://www.nature.com/articles/s41598-022-21366-2

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Bidirectional, LSTM, SimpleRNN, Input, GRU
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding, Dropout, Activation, MaxPooling1D,Conv1D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dense, BatchNormalization, LeakyReLU
from tensorflow.keras.initializers import glorot_uniform
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,roc_auc_score
import sklearn.metrics as metrics
from keras import models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_train= "/content/drive/MyDrive/CBiLSuccSite/Data/Train/"
path_test = "/content/drive/MyDrive/CBiLSuccSite/Data/Test/"

path_result = "/content/drive/MyDrive/CBiLSuccSite/Result/"
path_model ="/content/drive/MyDrive/CBiLSuccSite/Model/"

filename_train = "train_data.csv"
filename_test = "test_data.csv"

In [ ]:
file_train = filename_train
file_test = filename_test
df_train = pd.read_csv(path_train + file_train,  delimiter= ',')
df_test= pd.read_csv(path_test + filename_test,  delimiter= ',')

In [ ]:
df_train

,id,Sequence,Label
0,sp|P75863|X|344,CGSCRVQLLEGEVTPLKKSAMGDDGTILCCSCV,1
1,sp|P0ABS8|X|7,XXXXXXXXXXMLKNLAKLDQTEMDKVNVDLAAA,1
2,sp|P0ABS8|X|74,AHRLASVNLSRLPYEPKLKXXXXXXXXXXXXXX,1
3,sp|P0A7D7|X|3,XXXXXXXXXXXXXXMQKQAELYRGKAKTVYSTE,1
4,sp|P0A7D7|X|52,DGARIEQFDRKGMVNNKFNYFIMSKLAEAGIPT,1
...,...,...,...
9494,sp|P28331|X|355,AAIAGGLVDAEALVALKDLLNRVDSDTLCTEEV,0
9495,tr|I6X9E2|X|62,SDADWKPVADILGRTGKLNDGSVYKIGFARSDL,0
9496,sp|Q09666|X|4272,KISMPDFDLHLKGPKVKGDVDVSLPKVEGDLKG,0
9497,sp|F8VTL3|X|552,DEECWFPKATDKTFVEKLVQEQGSHSKFQKPRQ,0


In [ ]:
df_test

,id,Sequence,Label
0,sp|P09030|X|70,VFYHGQKGHYGVALLTKETPIAVRRGFPGDDEE,1
1,sp|P09030|X|24,INGLRARPHQLEAIVEKHQPDVIGLQETKVHDD,1
2,sp|P06996|X|31,VAGAANAAEVYNKDGNKLDLYGKVDGLHYFSDN,1
3,sp|P17854|X|102,TYRFIDELTDKLKLNLKVYRATESAAWQEARYG,1
4,sp|P17854|X|136,LWEQGVEGIEKYNDINKVEPMNRALKELNAQTW,1
...,...,...,...
502,sp|P34932|X|235,KLKVLATAFDTTLGGRKFDEVLVNHFCEEFGKK,0
503,sp|YFL022C|X|164,NSHLDSIDAKILNDLKKRKLIAQGKITDFNVTK,0
504,sp|P0A9Q5|X|8,XXXXXXXXXMSWIERIKSNITPTRKASIPEGVW,0
505,sp|YFL004W|X|568,RQDPKQAYEEEKKKLLKQKEIQKKIDGMRRLSN,0


In [ ]:
#def Dictionary for 1-gram
def Dic_1_gram():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
# Separation of words in protein sequence
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence

In [ ]:
k =1#1-gram
word_index1 =  Dic_1_gram()
vocab_size = len(word_index1)

In [ ]:
# Preprocessing for train data
texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())

MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = df_train['Label']
Xtrain.shape


(9499, 33)

In [ ]:
checkpoint = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1, mode='auto')

In [ ]:
lr = 0.0001
num_epochs = 80
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPool1D, Flatten, Dense,GlobalAveragePooling1D

from keras.layers import concatenate
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU, Flatten
def Classifier_CNN_BiLSTM(Xtrain,ytrain, Xtest, ytest):
    model1 = Sequential()
  #input
    model1.add(Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model1.add(Conv1D(64, 3, activation="relu", input_shape=(Xtrain.shape[1],1)))
    model1.add(MaxPool1D())
    model1.add(Dropout(0.4))
    model1.add(Conv1D(64,3, activation ="relu"))#,padding='valid',activation='relu',kernel_initializer='random_uniform', bias_initializer='zeros',strides=1))
    model1.add(MaxPool1D())
    model1.add(Bidirectional(GRU(16, dropout=0.4, recurrent_dropout=0,return_sequences=True)))
    model1.add(GlobalAveragePooling1D())
    model1.add(Flatten())


    model2 = Sequential()

    model2.add(Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model2.add(Dropout(0.4))
    model2.add(Bidirectional(LSTM(units=32,batch_input_shape=(None,TIME_STEPS, 300),return_sequences=True)))
    model2.add(Flatten())

    merged = concatenate(([model1.output, model2.output]))
    z = Dense(128, activation="relu")(merged)
    z = Dropout(0.2)(z)

    z = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=[model1.input, model2.input], outputs=z)

    model.compile(optimizer=keras.optimizers.Adam(learning_rate = lr),
                loss='binary_crossentropy',
                metrics=['accuracy',"AUC"])


    return model

In [ ]:
model = Classifier_CNN_BiLSTM(Xtrain, ytrain, Xtest,ytest)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 embedding_input (InputLaye  [(None, 33)]                 0         []                            
 r)                                                                                               
                                                                                                  
 embedding (Embedding)       (None, 33, 300)              6600      ['embedding_input[0][0]']     
                                                                                                  
 conv1d (Conv1D)             (None, 31, 64)               57664     ['embedding[0][0]']           
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 15, 64)               0         ['conv1d[0][0]']          

In [ ]:
result_CNN_BiLSTM = path_result + "CV_CNN_BiLSTM.txt"

In [ ]:
# Cross CNN_BiLSTM
num_folds =10
nb_filter = 64
kf = KFold(n_splits=num_folds, shuffle=True)
fold_scores = []
X = Xtrain
y = ytrain
# Lặp qua từng fold
for train_index, test_index in kf.split(X):# X1 physical feature, X2 NLP features
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    model = Classifier_CNN_BiLSTM(X_train, Y_train, X_test,Y_test)
    model.fit([X_train,X_train], Y_train, batch_size=32, epochs=num_epochs, validation_data = ([X_test,X_test],Y_test), shuffle = True,callbacks=[checkpoint],verbose=1)
    result1 = model.evaluate([X_test,X_test],Y_test)
    f = open(result_CNN_BiLSTM, 'a+', encoding='UTF-8')

    f.write("\n Cross-validation\n ")
    s = str(result1)
    f.write(s)
    f.close()
    y_pred= model.predict([X_test,X_test])
    ytest_true = Y_test
    y_pred = (y_pred > 0.5)

    y_pred = [int(i) for i in y_pred]

    y_pred = np.array(y_pred)
    y_true = np.array(ytest_true)

    cm = confusion_matrix(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)

    sn = cm[1][1]/(cm[1][1]+cm[1][0])
    sp = cm[0][0]/(cm[0][0]+cm[0][1])
    print(acc, mcc,sn,sp)
    print(cm)

    f = open(result_CNN_BiLSTM, 'a+', encoding='UTF-8')
    f.write("\n confusion_matric\n ")
    s = str(cm)
    f.write(s)
    f.close()

Epoch 1/80
268/268 [==============================] - 11s 16ms/step - loss: 0.6860 - accuracy: 0.5559 - auc: 0.5808 - val_loss: 0.6659 - val_accuracy: 0.6358 - val_auc: 0.6709
Epoch 2/80
268/268 [==============================] - 3s 10ms/step - loss: 0.6544 - accuracy: 0.6212 - auc: 0.6648 - val_loss: 0.6316 - val_accuracy: 0.6621 - val_auc: 0.7121
Epoch 3/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6264 - accuracy: 0.6510 - auc: 0.7069 - val_loss: 0.6038 - val_accuracy: 0.6695 - val_auc: 0.7431
Epoch 4/80
268/268 [==============================] - 3s 10ms/step - loss: 0.5995 - accuracy: 0.6814 - auc: 0.7436 - val_loss: 0.5807 - val_accuracy: 0.6905 - val_auc: 0.7742
Epoch 5/80
268/268 [==============================] - 3s 12ms/step - loss: 0.5683 - accuracy: 0.7086 - auc: 0.7774 - val_loss: 0.5503 - val_accuracy: 0.7295 - val_auc: 0.7995
Epoch 6/80
268/268 [==============================] - 3s 10ms/step - loss: 0.5497 - accuracy: 0.7175 - auc: 0.7935 - val_los

<ipython-input-20-f339f260d7d8>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/80
268/268 [==============================] - 11s 16ms/step - loss: 0.6850 - accuracy: 0.5582 - auc: 0.5847 - val_loss: 0.6701 - val_accuracy: 0.6021 - val_auc: 0.6358
Epoch 2/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6526 - accuracy: 0.6191 - auc: 0.6640 - val_loss: 0.6452 - val_accuracy: 0.6211 - val_auc: 0.6795
Epoch 3/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6263 - accuracy: 0.6517 - auc: 0.7067 - val_loss: 0.6290 - val_accuracy: 0.6547 - val_auc: 0.6998
Epoch 4/80
268/268 [==============================] - 3s 10ms/step - loss: 0.6053 - accuracy: 0.6722 - auc: 0.7351 - val_loss: 0.6071 - val_accuracy: 0.6695 - val_auc: 0.7340
Epoch 5/80
268/268 [==============================] - 3s 11ms/step - loss: 0.5785 - accuracy: 0.6976 - auc: 0.7659 - val_loss: 0.5843 - val_accuracy: 0.6958 - val_auc: 0.7635
Epoch 6/80
268/268 [==============================] - 3s 10ms/step - loss: 0.5557 - accuracy: 0.7121 - auc: 0.7877 - val_los

<ipython-input-20-f339f260d7d8>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/80
268/268 [==============================] - 12s 16ms/step - loss: 0.6831 - accuracy: 0.5604 - auc: 0.5898 - val_loss: 0.6621 - val_accuracy: 0.6189 - val_auc: 0.6513
Epoch 2/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6505 - accuracy: 0.6138 - auc: 0.6663 - val_loss: 0.6439 - val_accuracy: 0.6274 - val_auc: 0.6819
Epoch 3/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6281 - accuracy: 0.6524 - auc: 0.7036 - val_loss: 0.6238 - val_accuracy: 0.6474 - val_auc: 0.7092
Epoch 4/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6023 - accuracy: 0.6807 - auc: 0.7417 - val_loss: 0.5844 - val_accuracy: 0.6937 - val_auc: 0.7578
Epoch 5/80
268/268 [==============================] - 3s 11ms/step - loss: 0.5725 - accuracy: 0.7023 - auc: 0.7723 - val_loss: 0.5555 - val_accuracy: 0.7221 - val_auc: 0.7927
Epoch 6/80
268/268 [==============================] - 3s 11ms/step - loss: 0.5504 - accuracy: 0.7214 - auc: 0.7940 - val_los

...

<ipython-input-20-f339f260d7d8>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/80
268/268 [==============================] - 12s 18ms/step - loss: 0.6819 - accuracy: 0.5702 - auc: 0.5972 - val_loss: 0.6790 - val_accuracy: 0.5642 - val_auc: 0.5947
Epoch 2/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6531 - accuracy: 0.6163 - auc: 0.6612 - val_loss: 0.6531 - val_accuracy: 0.6158 - val_auc: 0.6609
Epoch 3/80
268/268 [==============================] - 3s 12ms/step - loss: 0.6280 - accuracy: 0.6506 - auc: 0.7065 - val_loss: 0.6294 - val_accuracy: 0.6295 - val_auc: 0.7043
Epoch 4/80
268/268 [==============================] - 3s 12ms/step - loss: 0.6027 - accuracy: 0.6776 - auc: 0.7406 - val_loss: 0.5986 - val_accuracy: 0.6695 - val_auc: 0.7399
Epoch 5/80
268/268 [==============================] - 3s 13ms/step - loss: 0.5718 - accuracy: 0.7064 - auc: 0.7731 - val_loss: 0.5726 - val_accuracy: 0.6874 - val_auc: 0.7706
Epoch 6/80
268/268 [==============================] - 3s 12ms/step - loss: 0.5504 - accuracy: 0.7218 - auc: 0.7935 - val_los

<ipython-input-20-f339f260d7d8>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]


Epoch 1/80
268/268 [==============================] - 12s 17ms/step - loss: 0.6854 - accuracy: 0.5613 - auc: 0.5876 - val_loss: 0.6670 - val_accuracy: 0.6196 - val_auc: 0.6660
Epoch 2/80
268/268 [==============================] - 3s 11ms/step - loss: 0.6507 - accuracy: 0.6166 - auc: 0.6674 - val_loss: 0.6317 - val_accuracy: 0.6396 - val_auc: 0.7010
Epoch 3/80
268/268 [==============================] - 3s 10ms/step - loss: 0.6181 - accuracy: 0.6588 - auc: 0.7178 - val_loss: 0.5992 - val_accuracy: 0.6913 - val_auc: 0.7526
Epoch 4/80
268/268 [==============================] - 3s 11ms/step - loss: 0.5851 - accuracy: 0.6885 - auc: 0.7576 - val_loss: 0.5679 - val_accuracy: 0.7123 - val_auc: 0.7796
Epoch 5/80
268/268 [==============================] - 3s 11ms/step - loss: 0.5630 - accuracy: 0.7116 - auc: 0.7804 - val_loss: 0.5519 - val_accuracy: 0.7229 - val_auc: 0.7937
Epoch 6/80
268/268 [==============================] - 3s 11ms/step - loss: 0.5481 - accuracy: 0.7192 - auc: 0.7942 - val_los

<ipython-input-20-f339f260d7d8>:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y_pred = [int(i) for i in y_pred]
